<h1>MUSHROOM CLASSIFICATION</h1>

<h2>Import Libaries</h2>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from copy import deepcopy

<h2>Load Dataset</h2>

In [2]:
df = pd.read_csv("dataset/mushrooms.csv")

In [3]:
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


<h2>Data Preparation</h2>

Copy dataset to a new DataFrame.

In [4]:
df2 = deepcopy(df)

 Replace the letters of the dataset with integers.

In [5]:
for column in df2.columns:
    distinct_elements = tuple(set(df2[column]))
    n = len(distinct_elements)
    for i in range(n):
        df2[column][df2[column] == distinct_elements[i]] = i

In [6]:
df2

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,1,3,0,0,7,1,1,1,0,...,3,3,3,0,1,1,3,0,4,4
1,1,1,3,2,0,5,1,1,0,0,...,3,3,3,0,1,1,3,1,0,5
2,1,3,3,3,0,2,1,1,0,1,...,3,3,3,0,1,1,3,1,0,2
3,0,1,0,3,0,7,1,1,1,1,...,3,3,3,0,1,1,3,0,4,4
4,1,1,3,8,1,0,1,0,0,0,...,3,3,3,0,1,1,2,1,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,1,0,3,0,1,0,0,1,0,2,...,3,7,7,0,3,1,3,5,1,1
8120,1,1,3,0,1,0,0,1,0,2,...,3,7,7,0,2,1,3,5,5,1
8121,1,5,3,0,1,0,0,1,0,1,...,3,7,7,0,3,1,3,5,1,1
8122,0,0,0,0,1,3,1,1,1,4,...,1,3,3,0,1,1,2,3,5,1


Change data type to integers, to avoid ValueError in SVC.

In [7]:
df2 = df2.astype('int32')

In [8]:
df2.dtypes

class                       int32
cap-shape                   int32
cap-surface                 int32
cap-color                   int32
bruises                     int32
odor                        int32
gill-attachment             int32
gill-spacing                int32
gill-size                   int32
gill-color                  int32
stalk-shape                 int32
stalk-root                  int32
stalk-surface-above-ring    int32
stalk-surface-below-ring    int32
stalk-color-above-ring      int32
stalk-color-below-ring      int32
veil-type                   int32
veil-color                  int32
ring-number                 int32
ring-type                   int32
spore-print-color           int32
population                  int32
habitat                     int32
dtype: object

In [9]:
y = df2['class']
X = df2.drop(columns=['class'])

Split data into train, test and validation sets.

In [10]:
X_train, X_test_and_validation, y_train, y_test_and_validation = train_test_split(X, y, train_size=0.6, random_state=42)

X_test, X_validation, y_test, y_validation = train_test_split(X_test_and_validation, y_test_and_validation, train_size=0.35, random_state=42)

<h2>Modelling</h2>

In [11]:
model = SVC()

In [12]:
model.fit(X_train, y_train)

SVC()

<h2>Test and evalute the model</h2>

In [13]:
model.score(X_train, y_train)

0.9948707427164547

In [14]:
model.score(X_validation, y_validation)

0.9962139138665405

In [15]:
model.score(X_test, y_test)

0.9982409850483729

<h1>Accuracy is quite good</h1>